In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v6/fact/product_type_name=app/granularity=monthly/date=2020-02-29/

In [0]:

segment_by_product_sql = "s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-au_app.v2/fact/granularity=monthly/month=2019-03/"
# after_transform=spark.read.format("delta").load(usage_basic_unified_path)
segment_by_product_sql=spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-au_app.v2/fact").parquet(segment_by_product_sql)
segment_by_product_sql.printSchema()

In [0]:

usage_basic_unified_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v6/fact/product_type_name=domain/granularity=daily/"
# df1 = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v6/fact").parquet(usage_basic_unified_path).createOrReplaceTempView("test")
start_str="2020-05-01"
end_str="2020-05-02"
after_transform=spark.read.format("delta").load(usage_basic_unified_path).where("date>='{}'".format(start_str)).where("date<='{}'".format(end_str))
# after_transform=spark.read.format("delta").load(usage_basic_unified_path)
after_transform.groupBy("date").count().orderBy("date",ascending=False).show()
after_transform=after_transform.createOrReplaceTempView("test_unified")

# spark.sql("select count(distinct(product_id)) from test_unified  where est_active_users is not null and est_active_users <> 0").show()
# spark.sql("select * from test_unified  where est_active_users is not null and est_active_users <> 0 and unified_category_key=800000 limit 1").show()

after_transform_avg=spark.sql("""
    select distinct product_id,sum(est_usage_penetration)/2 AS est_usage_penetration
    from test_unified  
    where est_active_users is not null 
    and est_active_users <> 0 
    and country_code = 'US' 
    and device_code = 'ios-phone' 
    and unified_category_key=800000
    group by product_id
    order by est_usage_penetration desc
    """)
# after_transform_avg=spark.sql("""
#     select product_id,est_usage_penetration
#     from test_unified  
#     where est_active_users is not null 
#     and est_active_users <> 0 
#     and country_code = 'US' 
#     and device_code = 'ios-phone' 
#     and unified_category_key=800000
#     order by est_usage_penetration desc
#     """)
after_transform_avg=after_transform_avg.createOrReplaceTempView("after_transform_avg")    
    

spark.sql("""
    select product_id, est_usage_penetration
    from after_transform_avg 
    order by est_usage_penetration desc
    """).show(1000, False)

# spark.sql("select distinct country_code,device_code,date,product_id,market_code,granularity from test where est_active_users is not null and est_active_users <> 0").createOrReplaceTempView("unified_remove_category")
# spark.sql("select count(*) as line_count from unified_remove_category").show()
# spark.sql("select count(distinct country_code) as country_count from unified_remove_category limit 1").show()
# spark.sql("select * from test limit 10").show()

In [0]:
%%sh
# mw.category_m 
# select  app_id, rank, kpi, estimate 
#             from plproxy.execute_select(\$proxy\$
#                 SELECT app_id, rank, kpi, estimate 
#                 FROM mu.category_monthly
#                 where date = '2020-01-31' 
#                 and rank <= 1000 
#                 and category_id = 36
#                 and store_id = 143441 
#                 and device_id = 2001 \$proxy\$)
#  t (app_id bigint, rank integer, kpi smallint, estimate double precision) order by rank asc;

# select  domain_id, rank_est_usage_penetration,est_usage_penetration 
#         from plproxy.execute_select(\$proxy\$
#             SELECT domain_id, rank_est_usage_penetration,est_usage_penetration 
#             FROM mw.category_m 
#             where date = '2020-01-31' 
#             and rank_est_usage_penetration <= 1000 
#             and category_id = 800000
#             and country_code = 'us'
#             and device_code = 'ip' \$proxy\$)
# t (domain_id bigint, rank_est_usage_penetration integer, est_usage_penetration real) order by rank_est_usage_penetration asc;

# PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF 
# internal-aa-prod-usage-plproxy-internal-1640809782.us-east-1.elb.amazonaws.com
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-usage-plproxy-internal-1640809782.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF
            select  domain_id, rank_est_usage_penetration,est_usage_penetration 
            from plproxy.execute_select(\$proxy\$
                SELECT domain_id, rank_est_usage_penetration,est_usage_penetration 
                FROM mw.category_d 
                where date = '2020-06-02' 
                and rank_est_usage_penetration <= 1000 
                and category_id = 800000
                and country_code = 'us'
                and device_code = 'ip' \$proxy\$)
 t (domain_id bigint, rank_est_usage_penetration integer, est_usage_penetration real) order by rank_est_usage_penetration asc;


EOF

In [0]:

import pandas as pd
from pyspark.sql import functions as F
from datetime import datetime
from applications.db_check_v1.common.db_check_utils import query
from pyspark.sql import Row
from pyspark.sql.types import *
from conf.settings import PG_USAGE_HOSTS, PG_USAGE_NAME, PG_USAGE_ACCESS_ID, PG_USAGE_SECRET_KEY, \
    CITUS_USAGE_NAME, CITUS_USAGE_ACCESS_ID, CITUS_USAGE_HOSTS, CITUS_USAGE_SECRET_KEY

PLPROXY_DSN = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_USAGE_NAME,
        user=PG_USAGE_ACCESS_ID,
        host=PG_USAGE_HOSTS[0][0],
        password=PG_USAGE_SECRET_KEY,
        port=PG_USAGE_HOSTS[0][1]
    )
)

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


begin_date = datetime(2019, 1, 1)
end_date = datetime(2020, 9, 30)

DATE_GRANULARITY_MAPPINGLIST = {
    "daily": get_date_list(begin_date, end_date, "D"),
    "weekly": get_date_list(begin_date, end_date, "W-SAT"),
    "monthly": get_date_list(begin_date, end_date, "M")
}


DATE_GRANULARITY_MAPPINGLIST["weekly"].reverse()

# print(DATE_GRANULARITY_MAPPINGLIST["monthly"])

sql_template_a_x_a = """
select kpi, sum(count_a) as app_count, sum(sum_e) as sum_e from plproxy.execute_select_nestloop($proxy$ 
    select kpi, count(estimate) as count_a, sum(estimate) as sum_e
    from ca.app_monthly
    where 
        date ='{date}'
    group by kpi order by kpi desc
$proxy$) tpl (kpi smallint, count_a bigint, sum_e double precision ) group by kpi order by kpi desc ;
"""

sql_template = """
select sum(count) as count, sum(affinity_sum) as affinity_sum,  sum(up_sum) as up_sum from plproxy.execute_select_nestloop($proxy$ 
    select count(*) as count, sum(est_cross_product_affinity) as affinity_sum,  sum(est_cross_product_usage_penetration) as up_sum
    from mw.{db_name}_m
    where 
        date ='{date}'
$proxy$) tpl (count bigint, affinity_sum real,up_sum real ) ;

"""

city_level_sql = """select sum(count) as count, sum(af_sum_value) as af_sum from plproxy.execute_select_nestloop($proxy$ 
    select count(*) as count, sum(est_affinity) as af_sum_value
    from mu.city_app_w
    where 
        date ='{date}' and  est_average_active_users <> 0 and est_average_active_users is not null
$proxy$) tpl (count bigint, af_sum_value real) ;
"""

domain_basic_sql = """
select  domain_id,sum 
            from plproxy.execute_select($proxy$
                SELECT domain_id,sum(est_usage_penetration)/2 AS sum
                FROM mw.category_w 
                where date >= '2020-05-02'
                and date <= '2020-05-09'
                and category_id = 800000
                and country_code = 'us'
                and device_code = 'ip' 
                group by domain_id$proxy$)
t (domain_id bigint, sum double precision);
"""

from pyspark.sql.types import StructType, StructField, LongType

def get_plproxy_result():
    plproxy_result = []
    result = query(PLPROXY_DSN, domain_basic_sql)
    # distinct_domain_id = result[1][0]
    # for _r in result:
    #     plproxy_result.append(_r[0])
        
    df_data = [Row(domain_id=long(r[0]),sum=r[1]) for r in result]
    # print df_data[1]
    _schema =StructType([StructField("domain_id", LongType(), False),
                        StructField("sum", DoubleType(), False)])
    df_plproxy = spark.createDataFrame(data=df_data, schema=_schema)
    return df_plproxy
    
# def get_unified_data():
#     unified_source_path = "s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobileweb.basic.v4/fact/granularity=w/month=202005/date=2020-05-16"
#     spark.read.format("delta").load(unified_source_path).createOrReplaceTempView("test_unified")
#     spark.sql("select distinct domain_id from test_unified where  est_average_active_users <> 0 and est_average_active_users is not null order by domain_id asc").createOrReplaceTempView("unified_df_new")

    
def compare():
    df_plproxy=get_plproxy_result()
    get_unified_data()
    df_plproxy.createOrReplaceTempView("plproxy_df_new")
    spark.sql("""select * 
                from plproxy_df_new
                order by sum desc""").createOrReplaceTempView("domain_diff")
    #spark.sql("select count(1) from domain_diff").show(10, False)
    spark.sql("select * from domain_diff").show(1000, False)
    
    

# unified_source_path = "s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobileweb.basic.v4/fact/granularity=w/date=2020-05-16"
# spark.read.format("delta").load(unified_source_path).show(10)

compare()


In [0]:
%%sh
